In [65]:
import os
import sys
import time

import matplotlib.pyplot as plt
import numpy as np

import GCode
import GRBL


In [66]:
sys.path.append(os.path.abspath(".."))
from utils import picture

# Draw a Brick Pattern

Attempt at programmatically making a brick pattern.

All units in mm. ```1``` = ```1 mm```.

> Jenga is played with 54 wooden blocks. Each block is three times longer than its width, and one fifth as thick as its length 1.5 cm × 2.5 cm × 7.5 cm (0.59 in × 0.98 in × 2.95 in).

https://en.wikipedia.org/wiki/Jenga

In [67]:
# Standard brick dimensions.
BrickHeight = 65  # [mm]
BrickLength = 225  # [mm]
BrickDepth = 12.5  # [mm]
BrickRatio = 215 / 65  # [dimensionless]

In [68]:
# Jenga
BlockHeight = 25  # [mm]
BlockLength = 75  # [mm]

In [100]:
# Drawing configuration.
# How many rows of bricks to draw on the block.
N_BrickRows = 10  # [dimensionless]

# Dimensions of a 'brick' projected onto the block of wood.
H_Block_Brick = BlockHeight / N_BrickRows  # [mm]
L_Block_Brick = H_Block_Brick * BrickRatio  # [mm]


# Code:

In [101]:
flip = np.array([[1, 1], [1, 0]])
transform_tuple = (
    np.eye(2),  # Identity matrix, do nothing.
    np.eye(2),  # Do nothing, for debugging.
    flip,  # Flip the matrix, reduces travel time.
)
vertical_brick_lines_tuple = (
    np.arange(L_Block_Brick, BlockLength, L_Block_Brick),# Odd rows.
    np.arange(L_Block_Brick / 2, BlockLength, L_Block_Brick), # Even rows.
)
horizontal_brick_lines = np.linspace(0, BlockHeight, N_BrickRows, endpoint=False)

Lines parallel to the X-axis.
Separates rows of bricks.

In [102]:
horizontal_brick_lines

array([ 0. ,  2.5,  5. ,  7.5, 10. , 12.5, 15. , 17.5, 20. , 22.5])

Lines parallel to the Y-axis.

In [103]:
vertical_brick_lines_tuple

(array([ 8.26923077, 16.53846154, 24.80769231, 33.07692308, 41.34615385,
        49.61538462, 57.88461538, 66.15384615, 74.42307692]),
 array([ 4.13461538, 12.40384615, 20.67307692, 28.94230769, 37.21153846,
        45.48076923, 53.75      , 62.01923077, 70.28846154]))

In [104]:
BlockBrick=GCode.Program()
BlockBrick.lines=list()

for idx in range(1, len(horizontal_brick_lines)):
    # Top horizontal line that defines each 'brick'
    horizontal_brick_line = horizontal_brick_lines[idx]
    row_line_points = np.array(
        [[0, horizontal_brick_line], [BlockLength, horizontal_brick_line]]
    )
    # Transform to perform on the row points.
    transform = transform_tuple[np.mod(idx, 2)]
    
    row_line_points = np.matmul(transform, row_line_points)
    line_ = GCode.Line(
        points=row_line_points,
    )
    
    BlockBrick.lines.append(line_)

In [105]:
for idx in range(1, len(horizontal_brick_lines)):

    # Vertical brick line.
    vertical_brick_lines = vertical_brick_lines_tuple[np.mod(idx, len(vertical_brick_lines_tuple))]
    
    start_point_y = horizontal_brick_lines[idx - 1]
    end_point_y = horizontal_brick_lines[idx]
    
    for idx2, vertical_brick_line in enumerate(vertical_brick_lines):
        transform = transform_tuple[np.mod(idx2, 2)]
        column_line_points = (
            np.array(
                [
                    [vertical_brick_line, start_point_y],
                    [vertical_brick_line, end_point_y],
                ]
            ),
        )
        column_line_points = np.matmul(transform, column_line_points)
        line_ = GCode.Line(column_line_points)
        BlockBrick.lines.append(line_)
# Not drawing the last row of vertical lines.
idx+=1
vertical_brick_lines = vertical_brick_lines_tuple[np.mod(idx, len(vertical_brick_lines_tuple))]

start_point_y = horizontal_brick_lines[idx-1]
end_point_y = BlockHeight
        
for idx2, vertical_brick_line in enumerate(vertical_brick_lines):
    transform = transform_tuple[np.mod(idx2, 2)]
    column_line_points = (
        np.array(
            [
                [vertical_brick_line, start_point_y],
                [vertical_brick_line, end_point_y],
            ]
        ),
    )
    column_line_points = np.matmul(transform, column_line_points)
    line_ = GCode.Line(column_line_points)
    BlockBrick.lines.append(line_)

In [106]:
BlockBrick.lines

[Line<cut=75.0mm, F=300, S=150>,
 Line<cut=75.0mm, F=300, S=150>,
 Line<cut=75.0mm, F=300, S=150>,
 Line<cut=75.0mm, F=300, S=150>,
 Line<cut=75.0mm, F=300, S=150>,
 Line<cut=75.0mm, F=300, S=150>,
 Line<cut=75.0mm, F=300, S=150>,
 Line<cut=75.0mm, F=300, S=150>,
 Line<cut=75.0mm, F=300, S=150>,
 Line<cut=2.5mm, F=300, S=150>,
 Line<cut=2.5mm, F=300, S=150>,
 Line<cut=2.5mm, F=300, S=150>,
 Line<cut=2.5mm, F=300, S=150>,
 Line<cut=2.5mm, F=300, S=150>,
 Line<cut=2.5mm, F=300, S=150>,
 Line<cut=2.5mm, F=300, S=150>,
 Line<cut=2.5mm, F=300, S=150>,
 Line<cut=2.5mm, F=300, S=150>,
 Line<cut=2.5mm, F=300, S=150>,
 Line<cut=2.5mm, F=300, S=150>,
 Line<cut=2.5mm, F=300, S=150>,
 Line<cut=2.5mm, F=300, S=150>,
 Line<cut=2.5mm, F=300, S=150>,
 Line<cut=2.5mm, F=300, S=150>,
 Line<cut=2.5mm, F=300, S=150>,
 Line<cut=2.5mm, F=300, S=150>,
 Line<cut=2.5mm, F=300, S=150>,
 Line<cut=2.5mm, F=300, S=150>,
 Line<cut=2.5mm, F=300, S=150>,
 Line<cut=2.5mm, F=300, S=150>,
 Line<cut=2.5mm, F=300, S=150>,

In [107]:
BlockBrick.time

1197.777350267149

In [108]:
BlockBrick.dist

2935.554700534298

In [109]:
BlockBrick.generate_gcode()

In [110]:
BlockBrick

Program<lines=99>

In [111]:
cnc = GRBL.GRBL(port="/dev/cnc_3018")
cnc.reset()

(0, ['ok', '', "Grbl 1.1f ['$' for help]", 'ok'])

In [112]:
cnc.status

'<Idle|MPos:-255.900,-136.000,-1.000|Bf:15,127|FS:0,0|WCO:0.000,0.000,0.000>'

In [113]:
BlockBrick.machine=cnc

In [114]:
BlockBrick.run()

In [88]:
BlockBrick.lines[0]

Line<cut=75.0mm, F=300, S=150>

In [89]:
BlockBrick.lines[0].power

150

In [90]:
BlockBrick.lines[0].dynamic_power

True

In [ ]:
BlockBrick.run()